In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [17]:
from rtree import index

In [6]:
#1.读取现状用地
xzyd_file = 'D:/2Gis/Arcgis Packages/B_三调_更新城区_最新1223/v108/全市合并.gdb'
# xzyd_columns = ['geometry', 'xzdkID', '用地类别f']
xzyd = gpd.read_file(xzyd_file,layer='xzyd_c1_rjl')
print("用地图层CRS:", xzyd.crs)

用地图层CRS: EPSG:4528


In [10]:
#2.读取交通小区
zone_file = 'D:/2Gis/Arcgis Packages/C_现状规划用地与交通小区0805导出/commondata/交通小区/市区交通小区202407关联行政区大区.shp'
zone = gpd.GeoDataFrame.from_file(zone_file).to_crs('EPSG:4326')
zone.drop(['ZONEID', 'TAZ_ID'], axis = 1, inplace = True)

In [12]:
pd.set_option('Display.max_columns',None)
zone.head(2)

,ID,XZQID,TID,TIDB,xzqJD,xzq,geometry
0,67,17,T00001,T016,高虹镇,临安区,"POLYGON ((119.72725 30.33494, 119.71949 30.332..."
1,3508,17,T00002,T016,高虹镇,临安区,"POLYGON ((119.72982 30.3162, 119.72922 30.3163..."


In [19]:
# 读取两个面层数据
gdf1 = xzyd
gdf2 = zone

# 确保两个面层的坐标系一致
gdf1 = gdf1.to_crs(gdf2.crs)

# 创建 RTree 空间索引
idx = index.Index()
for idx2, row2 in gdf2.iterrows():
    idx.insert(idx2, row2.geometry.bounds)

# 创建一个新的列，用于存储最大相交面积的属性
gdf1['belong_TID'] = None
gdf1['belong_TIDB'] = None
gdf1['belong_jiedao'] = None
gdf1['belong_xzq'] = None

# 遍历 gdf1 中的每个面
for idx1, row1 in gdf1.iterrows():
    max_area = 0
    max_attribute = None
    
    # 使用 RTree 索引快速查找可能相交的 gdf2 面
    for idx2 in idx.intersection(row1.geometry.bounds):
        row2 = gdf2.iloc[idx2]
        # 计算相交面积
        intersection = row1.geometry.intersection(row2.geometry)
        if not intersection.is_empty:
            area = intersection.area
            # 如果当前相交面积大于最大面积，则更新最大面积和属性
            if area > max_area:
                max_area = area
                max_attribute1 = row2['TID']  # 替换为 gdf2 中需要赋值的属性列名
                max_attribute2 = row2['TIDB']
                max_attribute3 = row2['xzqJD']
                max_attribute4 = row2['xzq']
    
    # 将最大相交面积的属性赋值给 gdf1
    gdf1.at[idx1, 'belong_TID'] = max_attribute1
    gdf1.at[idx1, 'belong_TIDB'] = max_attribute2
    gdf1.at[idx1, 'belong_jiedao'] = max_attribute3
    gdf1.at[idx1, 'belong_xzq'] = max_attribute4

# 查看结果
print(gdf1.head())

                  BSM  地块编号  xzdkID  DLBM   DLMC XDLDM  XDLMC dlmc_12 现状地类代码  \
0  330111211000021113  None     0.0  0702  农村宅基地  0702  农村宅基地    None   None   
1  330111211000021395  None     1.0  0702  农村宅基地  0702  农村宅基地    教育用地   None   
2  330111211000021815  None     2.0  0702  农村宅基地  0702  农村宅基地  二类居住用地   None   
3  330111211000021902  None     3.0  0702  农村宅基地  0702  农村宅基地    None   None   
4  330111211000022349  None     4.0  0702  农村宅基地  0702  农村宅基地    None   None   

  现状地类名称 现状主地类代码 现状主地类名称 现状主地类编码     z_ydf2 用地类别f land_type1 land_type2  \
0   None    0703    None    None  0703农村宅基地    R农      农村宅基地      农村宅基地   
1   None    0703    None    None  0703农村宅基地    R农      农村宅基地      农村宅基地   
2   None    0703    None    None  0703农村宅基地    R农      农村宅基地      农村宅基地   
3   None    0703    None    None  0703农村宅基地    R农      农村宅基地      农村宅基地   
4   None    0703    None    None  0703农村宅基地    R农      农村宅基地      农村宅基地   

  LANDUSAGE  similarity  rjl  z_rjl  z_rjl2   ldl  jzmd  jzgd  xglx 

In [22]:
# 读取 Excel 文件中的所有 Sheet
all_sheets = pd.read_excel('D:/2Gis/land/land202406/分区分类人口岗位面积.xlsx', sheet_name=None)

# 初始化一个空列表，用于存储所有长表
long_tables = []

# 遍历所有 Sheet
for sheet_name, df in all_sheets.items():
    # 将宽表转换为长表
    long_table = df.melt(
        id_vars=['用地类型'],  # 保留的列
        var_name='行政区',     # 转换后的列名（原列名）
        value_name=sheet_name  # 转换后的列名（使用 Sheet 名称作为列名）
    )
    # 将长表添加到列表中
    long_tables.append(long_table)

# 初始化合并的长表
combined_long_table = long_tables[0]  # 以第一个长表为基础

# 逐个合并其他长表
for i in range(1, len(long_tables)):
    combined_long_table = pd.merge(
        combined_long_table,
        long_tables[i],
        on=['用地类型', '行政区'],  # 按照这两列合并
        how='outer'  # 使用外连接，确保所有数据都被保留
    )

# 查看合并后的长表
print("合并后的长表：")
print(combined_long_table)

合并后的长表：
    用地类型  行政区    现状岗位  现状人口    规划岗位  规划人口
0      A  上城区    40.0   NaN    40.0   NaN
1      A  临安区    40.0   NaN    40.0   NaN
2      A  临平区    40.0   NaN    40.0   NaN
3      A  余杭区    40.0   NaN    40.0   NaN
4      A  富阳区    40.0   NaN    40.0   NaN
..   ...  ...     ...   ...     ...   ...
195    W  拱墅区  1000.0   NaN  1000.0   NaN
196    W  滨江区  1000.0   NaN  1000.0   NaN
197    W  萧山区  1000.0   NaN  1000.0   NaN
198    W  西湖区  1000.0   NaN  1000.0   NaN
199    W  钱塘区  1000.0   NaN  1000.0   NaN

[200 rows x 6 columns]


In [30]:
xzyd.shape

(115795, 44)

In [34]:
gdf1.columns

Index(['BSM', '地块编号', 'xzdkID', 'DLBM', 'DLMC', 'XDLDM', 'XDLMC', 'dlmc_12',
       '现状地类代码', '现状地类名称', '现状主地类代码', '现状主地类名称', '现状主地类编码', 'z_ydf2', '用地类别f',
       'land_type1', 'land_type2', 'LANDUSAGE', 'similarity', 'rjl', 'z_rjl',
       'z_rjl2', 'ldl', 'jzmd', 'jzgd', 'xglx', 'bz', 'ssqx', '供地方式', '权属性质',
       '开发动态', 'intersect_point_count', 'buffer_point_count',
       'total_point_count', 'SHAPE_Length', 'SHAPE_Area', 'BUILDAREA',
       'allocated_buildarea', 'cal_rjl', 'ref_rjl2', 'z_rjl3', 'rjl_final',
       'jzmj_cal', 'geometry', 'belong_TID', 'belong_TIDB', 'belong_jiedao',
       'belong_xzq'],
      dtype='object')

In [39]:
# 遍历面层数据
for idx, row in gdf1.iterrows():
    land_type = row['用地类别f']
    district = row['belong_xzq']
    
    # 在长表中查找匹配的行
    match = combined_long_table[(combined_long_table['用地类型'] == land_type) & (combined_long_table['行政区'] == district)]
    
    # 如果找到匹配的行，填充单位面积数据
    if not match.empty:
        gdf1.at[idx, '现状岗位平均面积'] = match['现状岗位'].values[0]
        gdf1.at[idx, '现状人口平均面积'] = match['现状人口'].values[0]

In [41]:
# 增加 pop 字段，初始化为 NaN
gdf1['现状人口数'] = np.nan

# 计算 pop，考虑单位面积为 0 或空值的情况
gdf1['现状人口数'] = np.where(
    (gdf1['现状人口平均面积'] != 0) & (gdf1['现状人口平均面积'] .notna()),  # 条件：单位面积不为 0 且不为空
    gdf1['jzmj_cal'] / gdf1['现状人口平均面积'],  # 满足条件时的计算
    np.nan  # 不满足条件时赋值为 NaN
)

# 计算 emp，考虑单位面积为 0 或空值的情况
gdf1['现状岗位数'] = np.where(
    (gdf1['现状岗位平均面积'] != 0) & (gdf1['现状岗位平均面积'] .notna()),  # 条件：单位面积不为 0 且不为空
    gdf1['jzmj_cal'] / gdf1['现状岗位平均面积'],  # 满足条件时的计算
    np.nan  # 不满足条件时赋值为 NaN
)

In [43]:
gdf1.head(3)

,BSM,地块编号,xzdkID,DLBM,DLMC,XDLDM,XDLMC,dlmc_12,现状地类代码,现状地类名称,现状主地类代码,现状主地类名称,现状主地类编码,z_ydf2,用地类别f,land_type1,land_type2,LANDUSAGE,similarity,rjl,z_rjl,z_rjl2,ldl,jzmd,jzgd,xglx,bz,ssqx,供地方式,权属性质,开发动态,intersect_point_count,buffer_point_count,total_point_count,SHAPE_Length,SHAPE_Area,BUILDAREA,allocated_buildarea,cal_rjl,ref_rjl2,z_rjl3,rjl_final,jzmj_cal,geometry,belong_TID,belong_TIDB,belong_jiedao,belong_xzq,现状岗位平均面积,现状人口平均面积,现状人口数,现状岗位数
0,330111211000021113,None,0.0,0702,农村宅基地,0702,农村宅基地,None,None,None,0703,None,None,0703农村宅基地,R农,农村宅基地,农村宅基地,None,0.0,NaN,NaN,0.8,NaN,NaN,NaN,None,None,None,None,None,None,0.0,0.0,0.0,147.022556,1107.407079,NaN,0.0,0.0,0.8,0.8,0.8,885.925676,"MULTIPOLYGON Z (((119.87513 30.05201 0, 119.87...",T03057,T011,富春街道,富阳区,NaN,180.0,4.921809,NaN
1,330111211000021395,None,1.0,0702,农村宅基地,0702,农村宅基地,教育用地,None,None,0703,None,None,0703农村宅基地,R农,农村宅基地,农村宅基地,None,0.0,1.0,1.0,1.0,35.0,20.0,24.0,None,None,富阳区,None,None,None,0.0,0.0,0.0,75.420733,357.676503,NaN,0.0,0.0,1.0,1.0,1.0,357.676503,"MULTIPOLYGON Z (((119.88651 30.07774 0, 119.88...",T03033,T011,富春街道,富阳区,NaN,180.0,1.987092,NaN
2,330111211000021815,None,2.0,0702,农村宅基地,0702,农村宅基地,二类居住用地,None,None,0703,None,None,0703农村宅基地,R农,农村宅基地,农村宅基地,None,0.0,2.0,2.0,1.0,30.0,30.0,0.0,None,规划,富阳区,None,None,None,0.0,0.0,0.0,305.332846,4715.063575,NaN,0.0,0.0,1.0,1.0,1.0,4715.063575,"MULTIPOLYGON Z (((119.96971 30.00124 0, 119.96...",T00100,T014,春江街道,富阳区,NaN,180.0,26.194798,NaN


In [46]:
gdf1['现状人口数'].sum()

12223172.821290607

In [50]:
gdf1['现状岗位数'].sum()

6175083.395749711

In [53]:
gdf1.columns

Index(['BSM', '地块编号', 'xzdkID', 'DLBM', 'DLMC', 'XDLDM', 'XDLMC', 'dlmc_12',
       '现状地类代码', '现状地类名称', '现状主地类代码', '现状主地类名称', '现状主地类编码', 'z_ydf2', '用地类别f',
       'land_type1', 'land_type2', 'LANDUSAGE', 'similarity', 'rjl', 'z_rjl',
       'z_rjl2', 'ldl', 'jzmd', 'jzgd', 'xglx', 'bz', 'ssqx', '供地方式', '权属性质',
       '开发动态', 'intersect_point_count', 'buffer_point_count',
       'total_point_count', 'SHAPE_Length', 'SHAPE_Area', 'BUILDAREA',
       'allocated_buildarea', 'cal_rjl', 'ref_rjl2', 'z_rjl3', 'rjl_final',
       'jzmj_cal', 'geometry', 'belong_TID', 'belong_TIDB', 'belong_jiedao',
       'belong_xzq', '现状岗位平均面积', '现状人口平均面积', '现状人口数', '现状岗位数'],
      dtype='object')

In [59]:
gdf1['total_point_count'].sum()

11714899.0

In [69]:
pd.set_option('display.float_format', '{:.1f}'.format)

In [73]:
gdf1.pivot_table(index=["belong_xzq"],  values=["现状人口数",'total_point_count'], aggfunc=sum, margins=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7660\3445279435.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  gdf1.pivot_table(index=["belong_xzq"],  values=["现状人口数",'total_point_count'], aggfunc=sum, margins=True)


,total_point_count,现状人口数
belong_xzq,,
上城区,1418534.0,1675789.2
临安区,688546.0,996667.4
临平区,1134289.0,1040081.5
余杭区,1405704.0,1386747.5
富阳区,930102.0,1132102.2
拱墅区,1196997.0,1506124.7
滨江区,579998.0,705874.6
萧山区,2284097.0,2004961.5
西湖区,1225126.0,1204683.4


In [83]:
pd.set_option('display.max_rows',None)

In [85]:
gdf1.pivot_table(index=["belong_jiedao"],  values=["现状人口数",'total_point_count'], aggfunc=sum, margins=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7660\3690233061.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  gdf1.pivot_table(index=["belong_jiedao"],  values=["现状人口数",'total_point_count'], aggfunc=sum, margins=True)


,total_point_count,现状人口数
belong_jiedao,,
丁兰街道,163555.0,228200.3
万市镇,23795.0,23564.3
三墩镇,292458.0,314904.0
上塘街道,70776.0,125153.9
上官乡,9559.0,3893.0
下沙街道,221684.0,127506.3
东新街道,112807.0,158889.5
东洲街道,79949.0,89727.4
东湖街道,89486.0,83469.3


In [71]:
gdf1.pivot_table(index=["用地类别f"], columns = ['belong_xzq'], values=["现状人口数"], aggfunc=sum, margins=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7660\1141191403.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  gdf1.pivot_table(index=["用地类别f"], columns = ['belong_xzq'], values=["现状人口数"], aggfunc=sum, margins=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7660\1141191403.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  gdf1.pivot_table(index=["用地类别f"], columns = ['belong_xzq'], values=["现状人口数"], aggfunc=sum, margins=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7660\1141191403.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy

现状人口数                                                   \
belong_xzq       上城区      临安区       临平区       余杭区       富阳区       拱墅区   
用地类别f                                                                   
A                0.0      0.0       0.0       0.0       0.0       0.0   
A中小              0.0      0.0       0.0       0.0       0.0       0.0   
A办               0.0      0.0       0.0       0.0       0.0       0.0   
A医               0.0      0.0       0.0       0.0       0.0       0.0   
A教科              0.0      0.0       0.0       0.0       0.0       0.0   
A文体              0.0      0.0       0.0       0.0       0.0       0.0   
A高               0.0      0.0       0.0       0.0       0.0       0.0   
B                0.0      0.0       0.0       0.0       0.0       0.0   
B办               0.0      0.0       0.0       0.0       0.0       0.0   
B市               0.0      0.0       0.0       0.0       0.0       0.0   
B旅               0.0      0.0       0.0       0.0       0.0       0.0   
M                0.0      0.0       0.0       0.0       0.0       0.0   
M创               0.0      0.0       0.0       0.0       0.0       0.0   
R          1650999.3 682941.3  847595.5 1096004.7  753095.5 1482946.4   
RB           12312.1  30541.5   70258.2   73301.3   37410.5   20004.3   
R农           12470.2 275834.8  121598.0  205885.2  335474.9    3174.1   
R农服              0.0      NaN       NaN       NaN       NaN       NaN   
R别墅              7.6   7349.9     629.8   11556.3    6121.3       NaN   
R服               0.0      NaN       NaN       NaN       NaN       0.0   
W                0.0      0.0       0.0       0.0       0.0       0.0   
All        1675789.2 996667.4 1040081.5 1386747.5 1132102.2 1506124.7   

                                                             
belong_xzq      滨江区       萧山区       西湖区      钱塘区        All  
用地类别f                                                        
A               0.0       0.0       0.0      0.0        NaN  
A中小             0.0       0.0       0.0      0.0        NaN  
A办              0.0       0.0       0.0      0.0        NaN  
A医              0.0       0.0       0.0      0.0        NaN  
A教科             0.0       0.0       0.0      0.0        NaN  
A文体             0.0       0.0       0.0      0.0        NaN  
A高              0.0       0.0       0.0      0.0        NaN  
B               0.0       0.0       0.0      0.0        NaN  
B办              0.0       0.0       0.0      0.0        NaN  
B市              0.0       0.0       0.0      0.0        NaN  
B旅              0.0       0.0       0.0      0.0        NaN  
M               0.0       0.0       0.0      0.0        NaN  
M创              0.0       0.0       0.0      0.0        NaN  
R          684194.1 1506726.0 1108225.9 449569.6 10262298.2  
RB          12091.2   95042.6   26761.5  43425.5   421148.7  
R农           9386.6  400959.3   66055.6  76922.0  1507760.7  
R农服             NaN       NaN       0.0      NaN        NaN  
R别墅           202.7    2233.7    3640.4    223.8    31965.3  
R服              0.0       NaN       0.0      NaN        NaN  
W               0.0       0.0       0.0      0.0        NaN  
All        705874.6 2004961.5 1204683.4 570140.9 12223172.8

In [57]:
gdf1[['用地类别f','total_point_count','现状人口数']]

,用地类别f,total_point_count,现状人口数
0,R农,0.0,4.921809
1,R农,0.0,1.987092
2,R农,0.0,26.194798
3,R农,0.0,1.243047
4,R农,0.0,1.754266
...,...,...,...
115790,A教科,0.0,NaN
115791,R,389.0,375.749995
115792,R服,0.0,NaN
115793,R,0.0,6.486797
